In [5]:
from sentence_transformers import SentenceTransformer
import pandas as pd
from tensorflow.keras import (
    models,
    layers,
    regularizers
)

2024-05-01 09:23:02.981329: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-01 09:23:03.235482: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-01 09:23:04.231465: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [6]:
sentence_model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-mpnet-base-v2", device='cuda'
                                     )

In [8]:
data = pd.read_parquet("data/mlt_data_publications.parquet")

In [26]:
titles_map = {row.paperId:row.title for row in data.itertuples()}

In [29]:
related_pubs = [(titles_map[row.paperId], 'relatedWith', titles_map[row.source]) 
                for row in data.itertuples() if row.con_type!='base']

In [36]:
titles_embs = sentence_model.encode(list(titles_map.values()), show_progress_bar=True)

Batches:   0%|          | 0/412 [00:00<?, ?it/s]

In [39]:
titles_emb_map = {paperId:emb for paperId, emb in zip(titles_map, titles_embs)}

In [41]:
titles_embs[0].shape

(768,)

In [ ]:
embeddings = sentence_model.encode(df_reshaped['text_cln'].to_list(), show_progress_bar=True)

In [ ]:
# Iniciar el modelo MLP secuencial.
mlp_model = models.Sequential([
    # Aplanar la imagen para convertirla de una matriz 2D a un vector 1D.
    # El tamaño de entrada se ajusta al tamaño de las imágenes.

    # Primera capa densa con 1024 unidades.
    # BatchNormalization se utiliza para estabilizar el aprendizaje y se aplica antes de la activación.
    layers.Dense(1024),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.Dropout(0.05),  # Dropout para la regularización.

    # Segunda capa densa con 512 unidades, con normalización y activación ReLU.
    layers.Dense(512),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.Dropout(0.10),  # Un poco más de Dropout para esta capa más profunda.

    # Tercera capa densa con 256 unidades, y regularización L1 y L2 para evitar el sobreajuste.
    layers.Dense(256, kernel_regularizer=regularizers.l1_l2(l1=0.001, l2=0.001)),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.Dropout(0.15),

    # Cuarta capa densa con 128 unidades, también con regularización y BatchNormalization.
    layers.Dense(128, kernel_regularizer=regularizers.l1_l2(l1=0.01, l2=0.01)),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.Dropout(0.20),

    # Capa de salida con activación softmax para la clasificación multiclase.
    layers.Dense(NUM_CLASSES, activation='softmax')
])

# Compilar el modelo especificando el optimizador y la función de pérdida.
mlp_model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['categorical_accuracy'])

# Callback para guardar el mejor modelo durante el entrenamiento, basado en la pérdida de validación.
checkpoint_callback = callbacks.ModelCheckpoint(
    filepath='models/mlp_best.keras',  # Guarda el modelo completo.
    save_weights_only=False,
    monitor='val_loss',
    mode='min',
    save_best_only=True)  # Solo guarda el modelo si es el mejor hasta el momento.

# Entrenar el modelo utilizando un generador de datos y validación con un conjunto de datos separados.
history_mlp = mlp_model.fit(
    train_generator,
    validation_data=(X_val_p, y_val_p),
    epochs=100,  # Número de épocas para entrenar.
    callbacks=[checkpoint_callback]  # Incluye el checkpoint para el guardado del modelo.
)

# Cargar el mejor modelo guardado después del entrenamiento.
mlp_model = models.load_model('models/mlp_best.keras')

NameError: name 'df_reshaped' is not defined